## Outlier detection - Normalizing by volume for each subject

In [1]:
#Import the libraries needed
import pandas as pd
import glob
import numpy as np
import time
import re
import os
import xml.etree.ElementTree as et
import csv
from nilearn import image
from nilearn import plotting
import warnings
warnings.filterwarnings('ignore')

C:\Users\Muthulakshmi C\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
def create_iqr_dict(p,q1,q3,unique_roi):
    #Make a copy of the panel. 
    #Find IQR of each column for the specified quartiles. 
    #Check if each entry is in the IQR - output False if no outlier, True if it is an outlier. 
    p_new = p
    dict_iqr = {}
    iqr_dict = {}
    panel_actualval = pd.Panel()
    for i in range(len(unique_roi)) :
        each_roi = unique_roi[i]
        df = p_new[each_roi]
        Q1 = df.quantile(q1)
        Q3 = df.quantile(q3)
        IQR = Q3 - Q1
        df_new = (df < (Q1 - 1.5 * IQR))|(df > (Q3 + 1.5 * IQR))
        y = list(df_new[df_new == True].stack().index) 
        for element in y:
            val = df.loc[element[0],str(element[1])]
            df_new.loc[element[0],str(element[1])] = val
        p_new[each_roi] = df_new

        iqr_df = pd.DataFrame(df_new.columns)
        iqr_df = iqr_df.append(Q1)
        iqr_df = iqr_df.append(Q3)
        dict_iqr[each_roi] = iqr_df
        iqr_dict[each_roi] = [Q1,Q3]
        panel_iqr = pd.Panel(dict_iqr)
    return panel_iqr,iqr_dict      


In [3]:
def create_roi_dict():
    #Create a dictionary of ROI_IDs and their names from the .xml file
    roi_dict={}
    current_path = os.getcwd()
    xml_file = os.path.join(os.getcwd(),'labels.xml')
    root = et.parse(xml_file).getroot()
#     print('Root :',root)
    for roi in root.findall('label'):
        att = roi.attrib
        roi_id = int(att.get('id'))
        roi_name = att.get('fullname')
        roi_dict[roi_id] = roi_name

#     print('Attributes:',att)
#     print('ROI_ID:',roi_id)
#     print('ROI_Name:',roi_name)
    return roi_dict

In [4]:
def create_testdf(testfile):
    test_file = pd.read_csv(testfile,delimiter='\t')
    test_df = test_file[test_file['ROI_ID']!=900] #removing cerebellum
    test_df.drop(columns='CSF_Volume(mm^3)',inplace=True) #removing CSF_Volume
            #Normalizing by volume per subject across all rois
    tot_vol1 = test_df['Total_Volume(GM+WM)(mm^3)'].sum()
    #     print(tot_vol)
    for item in test_df.columns:
        if(item!='ROI_ID'):
            test_df.loc[:,str(item)] /= tot_vol1
    return test_df

In [5]:
def write_to_file(output_file,df):
    df.to_csv(output_file, sep=',',index=False)

In [6]:
def test_outliers(panel_iqr,test_df,quart1,quart3,common_roi,iqr_dict):
    outlier_roi=[]
#     output_file = 'output_iqr_116_S_6517_T1w_edited.txt'
#     with open(output_file,'w') as file:
#     #         print('File:',allFiles_new[i])
    c=0
    outlier_df = pd.DataFrame(columns=['LowerBound','UpperBound','ROI_ID','ROI_Name','Measure','Value','InlierBound1','InlierBound2'])
    roi_dict = create_roi_dict()
    for every_roi in common_roi :
        all_stat = test_df[test_df['ROI_ID'] == every_roi]
        roi_stat = all_stat.drop('ROI_ID',axis = 1, inplace=False)
        iqr_info = iqr_dict[every_roi]
        q1 = iqr_dict[every_roi][0]
        q3 = iqr_dict[every_roi][1]
        iqr = q3 - q1
        temp =(roi_stat < (q1 - 1.5 * iqr))|(roi_stat > (q3 + 1.5 * iqr)) 
        temp2 = pd.DataFrame(data = temp)
        s = temp2.stack()
        outlier_regions = s[s].index.tolist()
        if(len(outlier_regions)!=0):
            outlier_roi.append(every_roi)
            for item in outlier_regions :
                roi_id = every_roi
                roi_name = roi_dict[roi_id]
                measure = item[1]
                val = all_stat.loc[item[0],str(item[1])]
#                     print(q1.type)
                bound1 = panel_iqr[roi_id].loc[float(quart1),str(measure)]
                bound2 = panel_iqr[roi_id].loc[float(quart3),str(measure)]
                outlier_row=[quart1,quart3,roi_id,roi_name,measure,val,bound1,bound2]
                outlier_df = outlier_df.append(pd.Series(dict(zip(outlier_df.columns,outlier_row))), ignore_index=True)
#                 print(roi_id,"-",roi_name,"-",measure,"-",val,"-",bound1,"-",bound2)
#                 file.write('['+str(quart1)+','+str(quart3)+']\t'+str(roi_id)+'\t'+str(roi_name)+'\t'+str(measure)+'\t'+str(val)+'\t[ '+str(bound1)+' - '+str(bound2)+' ]\n')
    return outlier_df,outlier_roi

In [9]:
#Paths of input and output files
#path - the path of input file 
#path_iqr - path of output file
current_dir = os.getcwd()
path = os.path.join(current_dir,'cnstats')
path_iqr = os.path.join(current_dir,'cnstats_iqr')

In [10]:
#Output file directory 
#Removing all files already in the directory
opdir_avl_files = glob.glob(path_iqr+"/*.txt")
for f in opdir_avl_files:
    os.remove(f)

In [11]:
allFiles = glob.glob(path + "/*.txt")

#Reading each file as a dataframe, and storing it in a list
fileslist = []
frame = pd.DataFrame()
for i in range(len(allFiles)):
    file = allFiles[i]
    df = pd.read_csv(file,index_col=None, header=0,delimiter='\t')
    df = df[df['ROI_ID']!=900] #removing cerebellum
    df.drop(columns='CSF_Volume(mm^3)',inplace=True) #removing CSF_Volume
    #Normalizing by volume per subject across all rois
    tot_vol = df['Total_Volume(GM+WM)(mm^3)'].sum()
    for item in df.columns:
        if(item!='ROI_ID'):
            df.loc[:,str(item)] /= tot_vol
    fileslist.append(df)

In [12]:
#Convert the list into a vertical stack, and find the column headers
data_2d_array = np.vstack(fileslist)
data_vstack = pd.DataFrame(data_2d_array)
# print(data_vstack.shape)
data_vstack.columns = df.columns.values.tolist()

In [13]:
#Find the unique ROI_IDs and store it 
unique_roi = data_vstack.ROI_ID.unique()
unique_roi = [int(i) for i in unique_roi]

In [14]:
#3D dataframe to be implemented using Panels. Hence reordering the index.
#Data is grouped by ROI_ID. Dict keys are ROI_IDs.
temp_df = pd.DataFrame()
temp_dict = {}
for i in range(len(unique_roi)):
    each_roi = unique_roi[i]
    temp_df = data_vstack[data_vstack['ROI_ID'] == each_roi]
    index_range = range(temp_df.shape[0])
    temp_df['new_ind'] = index_range
    temp_df.set_index('new_ind',inplace=True)
    temp_dict[each_roi] = temp_df
#     print(temp_df)

In [15]:
#Create a panel from the dictionary
p = pd.Panel(temp_dict)
# print(p)

In [16]:
tf = '116_S_6517_T1w_edited.roiwise.stats.txt'
test_df = create_testdf(tf)

In [17]:
unique_roi_test = list(test_df['ROI_ID'])
# print(unique_roi_test)

In [18]:
common_roi = list(set(unique_roi).intersection(unique_roi_test))

In [19]:
# panel_iqr = pd.Panel(dict_iqr)
iqr_limits = [[0.1,0.9]]
outlier_df = pd.DataFrame()
outlier_roi = []
#Create IQR dictionary
for i in range(len(iqr_limits)):
    i1 = iqr_limits[i]
    limit1 = i1[0]
    limit2 = i1[1]
#     print("Limits:",limit1,"-",limit2)
    p = pd.Panel(temp_dict)
    panel_iqr,iqr_dict = create_iqr_dict(p,limit1,limit2,unique_roi)
    g,roi = test_outliers(panel_iqr,test_df,limit1,limit2,common_roi,iqr_dict)
    outlier_roi.append(roi)
    outlier_df = outlier_df.append(g,ignore_index=True)
#     outf = 'output_iqr_116_S_6517_T1w_edited.txt'
#     print(g)
    
    

In [21]:
#Write to output file 
output_file = 'output_iqr_116_S_6517_T1w_edited.csv'
write_to_file(output_file,outlier_df)

In [22]:
#Open the actual image, and then the outlier image
path = os.path.join(current_dir,'116_S_6517_T1w_edited.svreg.label.nii.gz')
labeldata = image.load_img(path)
# print(labeldata.shape)
# plotting.plot_stat_map(labeldata)

In [23]:
ld = labeldata.get_data()
x,y,z = np.shape(ld)
for i in range(x):
    for j in range(y):
        for k in range(z):
            if ld[i,j,k]>1000:
                ld[i,j,k]=ld[i,j,k]%1000
                

In [24]:
# lda = labeldata.get_data()
arr = np.zeros(shape=np.shape(ld))
for i in range(len(outlier_roi)):
    if i==0:
        new_elem = outlier_roi[i]
    else:
        list1 = outlier_roi[i-1]
        list2 = outlier_roi[i]
        new_elem = list(np.setdiff1d(list2,list1))
#     print('------------')
#     print('New elem:',new_elem)
    lda = np.isin(ld, new_elem)
#     print("Iter:",i," before :",np.count_nonzero(arr))    
    arr += lda.astype(int) * (80*(i+1))
#     print("Iter:",i," aft :",np.count_nonzero(arr))
#     print(np.isclose(arr,80*(i+1)).sum())

In [25]:
new_lda = image.new_img_like(labeldata, arr, affine=None, copy_header=True)
# plotting.plot_stat_map(new_lda,cmap='gray')

In [26]:
new_lda.to_filename('output_iqr_116_S_6517_T1w_edited.nii.gz')